In [28]:
import typing
import queue
import dataclasses

In [29]:
@dataclasses.dataclass(order=True)
class Item:
    priority: int = 0


In [34]:
import collections
@dataclasses.dataclass
class DevinPriorityQueue:
    queues: typing.Dict[float, collections.deque[Item]] = dataclasses.field(default_factory=dict)
    #priorities: typing.List[float] = dataclasses.field(default_factory=list)
    #current_lowest_priority: float = None
    current_priority: typing.Optional[float] = None
    ct: int = 0
    
    def put(self, item: Item, priority: float):
        # NOTE: adding a priority should be rare
        if priority not in self.queues:
            self.queues[priority] = collections.deque()
            self.queues = {k:v for k,v in sorted(self.queues.items(), key=lambda x: x[0])}
            
        self.queues[priority].appendleft(item)
        self.ct += 1
        
        # the queue will have at least one element
        if self.current_priority is None or priority < self.current_priority:
            self.current_priority = priority
    
    def get_all(self) -> typing.List[Item]:
        '''Get all items in the queue, sorted by priority.'''
        return [v for q in self.queues.values() for v in q]
    
    def get(self) -> Item:
        if self.current_priority is None:
            raise IndexError('Cannot pop from empty queue')
        cq = self.current_queue
        v = cq.pop()
        self.ct -= 1
        
        if not len(cq):
            self.current_priority = self.get_lowest_priority()
        
        return v
    
    @property
    def current_queue(self) -> collections.deque:
        try:
            return self.queues[self.current_priority]
        except KeyError:
            raise IndexError('Cannot pop from empty queue') from e
        
    def get_lowest_priority(self) -> typing.Optional[float]:
        '''Get the lowest priority queue that has items.'''
        for p,q in self.queues.items():
            if len(q):
                return p
        return None
    
    def empty(self) -> bool:
        return self.ct == 0
            
    def size(self) -> int:
        return self.ct

In [39]:
def assert_devin(items: typing.List[Item]):
    q = DevinPriorityQueue()
    for i in items:
        q.put(i, i.priority)
    assert([q.get() for _ in range(q.size())] == list(sorted(items)))

def put_tester_devin(q: queue.PriorityQueue, items: typing.List[Item]):
    for i in items:
        q.put(i, i.priority)

def put_tester_builtin(q: queue.PriorityQueue, items: typing.List[Item]):
    for i in items:
        q.put(i)

def get_tester(q):
    while not q.empty():
        q.get()
    return q

test1 = [Item(1) for _ in range(100)]
test2 = [Item(i) for i in range(100)]
test3 = [Item(j) for i in range(100) for j in [i]*10]
all_tests = test1 + test2 + test3

# MAKE SURE DEVIN IS WORKING WELL!!
assert_devin(all_tests)

dq = DevinPriorityQueue()
bq = queue.PriorityQueue()
%timeit put_tester_devin(dq, test1)
%timeit put_tester_builtin(bq, test1)
%timeit put_tester_devin(dq, test2)
%timeit put_tester_builtin(bq, test2)
%timeit put_tester_devin(dq, test3)
%timeit put_tester_builtin(bq, test3)

%timeit get_tester(dq)
%timeit get_tester(bq)
%timeit get_tester(dq)
%timeit get_tester(bq)
%timeit get_tester(dq)
%timeit get_tester(bq)

63.8 µs ± 661 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
297 µs ± 7.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
66 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
302 µs ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
640 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.12 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
668 ns ± 401 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.36 µs ± 525 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)
321 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
855 ns ± 29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
330 ns ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
846 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


638 ns ± 436 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.48 µs ± 439 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)
300 ns ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
819 ns ± 3.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
280 ns ± 3.33 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
820 ns ± 4.01 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
